In [1]:
import sys
sys.path.append('../')

import tquant as tq
from engine.globalconfig import GlobalConfig
from engine.loader.curveload import MarketDataLoader

from datetime import date, timedelta
import tensorflow as tf
import pandas as pd

import QuantLib as ql

In [2]:
config = GlobalConfig.from_json()
mdl = MarketDataLoader(config)
tq.Settings.evaluation_date

datetime.date(2023, 9, 29)

In [3]:
calendar = tq.TARGET()
daycounter = tq.DayCounter(tq.DayCounterConvention.Actual360)
evaluation_date = tq.Settings.evaluation_date

In [4]:
rates = mdl.ir_eur_curve_estr['quote'].values
times = mdl.ir_eur_curve_estr['daycount'].values/365

In [5]:
today = ql.Date(29,9,2023)
ql.Settings.instance().evaluationDate = today

sp = [1.0, 0.9941, 0.9826, 0.9674, 0.9488, 0.9246, 0.8945, 0.8645, 0.83484, 0.80614, 0.7784]
dp = [1 - s for s in sp]
ql_times = [today + int(day*365) for day in times]
ql_times_sp = [today + int(n*365) for n in range(11)]
times_sp = [0 + int(n) for n in range(11)]

estr_curve = tq.RateCurve(times, rates)
default_curve = tq.SurvivalProbabilityCurve(times_sp, sp)

In [6]:
ql_crv = ql.SurvivalProbabilityCurve(ql_times_sp, sp, ql.Actual365Fixed(), ql.TARGET())
ql_crv.enableExtrapolation()

ql_estr_curve = ql.ZeroCurve(ql_times,rates,ql.Actual365Fixed())
ql_estr_handle = ql.RelinkableYieldTermStructureHandle()
ql_estr_handle.linkTo(ql_estr_curve)

In [7]:
probability = ql.DefaultProbabilityTermStructureHandle(ql_crv)
recoveryRate = 0.4
integralStep = ql.Period('1d')
engine = ql.IntegralCdsEngine(integralStep,probability, recoveryRate, ql_estr_handle)


In [8]:
spread = 34.6 / 10000
nominal = 1000000

cdsSchedule = ql.MakeSchedule(ql.Date(22,3,2024), 
                              ql.Date(22, 3, 2027), 
                              ql.Period('3M'),
                              ql.Quarterly, 
                              ql.TARGET(), 
                              ql.Following
                              )

cds = ql.CreditDefaultSwap(ql.Protection.Seller, nominal, spread, cdsSchedule, ql.Following, ql.Actual365Fixed())

cds.setPricingEngine(engine)

print(cds.NPV())
print(cds.couponLegNPV())
print(cds.defaultLegNPV())

-12026.923702065273
9474.042901271438
-21500.96660333671


In [9]:
# --------------
# Premium leg
start_date = date(2024, 3, 22)
fixed_schedule = [start_date]
fixed_schedule.extend([start_date + timedelta(365 *.25 *i) for i in range(1,13)])
fixed_spread = [34.6 / 10000]*len(fixed_schedule)
fixed_notional = [1000000]*len(fixed_schedule)
fixed_daycounter = tq.DayCounter(tq.DayCounterConvention.Actual365)
swap_type = tq.SwapType.Payer

premium_leg = tq.PremiumLeg(fixed_schedule, 
                            fixed_notional, 
                            fixed_spread, 
                            fixed_daycounter,
                            tq.CompoundingType.Simple, 
                            tq.Frequency.Quarterly
                            )
# --------------
# floating leg
default_schedule = [start_date]
default_schedule.extend([start_date + timedelta(365 *.25 *i) for i in range(1,13)])

floating_notional = [1000000]*len(default_schedule)
floating_daycounter = tq.DayCounter(tq.DayCounterConvention.Actual365)

default_leg = tq.DefaultLeg(default_schedule, 
                            floating_notional, 
                            recoveryRate, 
                            floating_daycounter,
                            tq.CompoundingType.Simple, 
                            tq.Frequency.Quarterly
                            )
# --------------
# cds object

cds = tq.CreditDefaultSwap.from_legs(swap_type, 
                           premium_leg,
                           default_leg)

cds_engine = tq.CDSAnalyticEngine(cds)

npv_cds = cds_engine.price(estr_curve, default_curve, evaluation_date)
print(npv_cds.numpy())

-12062.8282329174


In [10]:
print(fixed_schedule)

[datetime.date(2024, 3, 22), datetime.date(2024, 6, 21), datetime.date(2024, 9, 20), datetime.date(2024, 12, 20), datetime.date(2025, 3, 22), datetime.date(2025, 6, 21), datetime.date(2025, 9, 20), datetime.date(2025, 12, 20), datetime.date(2026, 3, 22), datetime.date(2026, 6, 21), datetime.date(2026, 9, 20), datetime.date(2026, 12, 20), datetime.date(2027, 3, 22)]


In [11]:
print(estr_curve.discount(1))
print(ql_estr_curve.discount(ql.Date(29,9,2024)))

tf.Tensor(0.9617597256015296, shape=(), dtype=float64)
0.9618675656625507


In [12]:
print(evaluation_date)
print(today)
print(default_curve.survival_probability(tq.date(2026,9,29), 
                                         tq.DayCounter(tq.DayCounterConvention.Actual365), 
                                         evaluation_date))
for p,d in zip (default_curve.survival_probabilities,default_curve.pillars):
    print(p,d)
print(ql_crv.survivalProbability(ql.Date(29,9,2026)))

2023-09-29
September 29th, 2023
tf.Tensor(0.9673485462048487, shape=(), dtype=float64)
tf.Tensor(1.0, shape=(), dtype=float64) 0
tf.Tensor(0.9941, shape=(), dtype=float64) 1
tf.Tensor(0.9826, shape=(), dtype=float64) 2
tf.Tensor(0.9674, shape=(), dtype=float64) 3
tf.Tensor(0.9488, shape=(), dtype=float64) 4
tf.Tensor(0.9246, shape=(), dtype=float64) 5
tf.Tensor(0.8945, shape=(), dtype=float64) 6
tf.Tensor(0.8645, shape=(), dtype=float64) 7
tf.Tensor(0.83484, shape=(), dtype=float64) 8
tf.Tensor(0.80614, shape=(), dtype=float64) 9
tf.Tensor(0.7784, shape=(), dtype=float64) 10
0.9673490410958905


In [13]:
default_leg.display_flows()

,start_period,end_period,payment_date,notional,recovery,accrual,day_counter
0,2024-03-22,2024-03-23,2024-03-23,1000000,0.4,0.00274,Actual365
0,2024-03-23,2024-03-24,2024-03-24,1000000,0.4,0.00274,Actual365
0,2024-03-24,2024-03-25,2024-03-25,1000000,0.4,0.00274,Actual365
0,2024-03-25,2024-03-26,2024-03-26,1000000,0.4,0.00274,Actual365
0,2024-03-26,2024-03-27,2024-03-27,1000000,0.4,0.00274,Actual365
...,...,...,...,...,...,...,...
0,2027-03-17,2027-03-18,2027-03-18,1000000,0.4,0.00274,Actual365
0,2027-03-18,2027-03-19,2027-03-19,1000000,0.4,0.00274,Actual365
0,2027-03-19,2027-03-20,2027-03-20,1000000,0.4,0.00274,Actual365
0,2027-03-20,2027-03-21,2027-03-21,1000000,0.4,0.00274,Actual365
